In [2]:
import pandas as pd
import numpy as np
import os
import gc
import time
from tqdm import tqdm
import subprocess
import matplotlib.pyplot as plt
import seaborn as sns

In [54]:
df_test = pd.read_csv('/home/c5/Рабочий стол/Модуль 4/BIG DATA/df_0.csv', nrows=1000)
print("\nТекущие размеры столбцов:")
print(df_test.memory_usage(deep=True))


Текущие размеры столбцов:
Index     132
0        8000
1        8000
2        8000
3        8000
4        8000
5        8000
6        8000
7        8000
8        8000
9        8000
10       8000
11       8000
12       8000
13       8000
14       8000
15       8000
16       8000
17       8000
18       8000
19       8000
20       8000
21       8000
22       8000
23       8000
24       8000
25       8000
26       8000
27       8000
28       8000
29       8000
dtype: int64


посмотрим количество записей в первом набора данных df_0

In [65]:
data_dir = '/home/c5/Рабочий стол/Модуль 4/BIG DATA'

files = [
    'df_0.csv', 'df_1.csv', 'df_2.csv', 'df_3.csv',
    'target_0.csv', 'target_1.csv', 'target_2.csv', 'target_3.csv',
    'df_val.csv', 'target_val.csv'
]


for file_name in files:
    file_path = os.path.join(data_dir, file_name)
    df_sample = pd.read_csv(file_path, nrows=0)
    num_columns = len(df_sample.columns)
    result = subprocess.run(['wc', '-l', file_path], stdout=subprocess.PIPE)
    num_rows = int(result.stdout.decode().split()[0]) - 1  # Вычитаем 1 для заголовка
    
    print(f"{file_name}: Shape: ({num_rows}, {num_columns})")

df_0.csv: Shape: (30000000, 30)
df_1.csv: Shape: (30000000, 30)
df_2.csv: Shape: (30000000, 30)
df_3.csv: Shape: (30000000, 30)
target_0.csv: Shape: (30000000, 2)
target_1.csv: Shape: (30000000, 2)
target_2.csv: Shape: (30000000, 2)
target_3.csv: Shape: (30000000, 2)
df_val.csv: Shape: (30000000, 30)
target_val.csv: Shape: (30000000, 2)


Определяет минимальный возможный тип данных для серии

In [76]:
def get_min_numeric_type(series):
    if series.dtype.kind == 'i':
        return pd.Int32Dtype()  
    elif series.dtype.kind == 'f':
        return 'float32' 
    else:
        return series.dtype  

Оптимизирует типы данных во всем датафрейме ( до минимального )

In [77]:
def optimize_dataframe_types(df):
    for col in df.columns:
        if pd.api.types.is_numeric_dtype(df[col]):
            df[col] = df[col].astype(get_min_numeric_type(df[col]))
    return df

Загрузим большой CSV файл по частям, разделив на чанки, попробуем оптимизировать память

In [78]:
def load_large_csv(file_path, chunksize=500000):
    combined_df = []
    start_time = time.time()  # Начало отсчета времени
    
    generator = pd.read_csv(file_path, chunksize=chunksize, low_memory=False, on_bad_lines='warn')
    for i, chunk in enumerate(generator):
        print(f"Обрабатываем чанк {i + 1} из файла {os.path.basename(file_path)}")
        chunk = optimize_dataframe_types(chunk)  # Оптимизация типов данных
        combined_df.append(chunk)
    
    combined_df = pd.concat(combined_df, ignore_index=True)
    
    end_time = time.time()  # Конец отсчета времени
    print(f"Время загрузки {os.path.basename(file_path)}: {end_time - start_time:.2f} секунд")
    
    return combined_df

Загружаем по чангам и обрабатываем по частям: df_0, target_0,......,df_val,target_val

In [79]:
def load_and_process_files(data_dir):
    dfs = []  # Список для хранения обучающих DataFrame
    targets = []  # Список для хранения целевых переменных
    
    for i in range(4):
        print(f"\nЗагрузка файла df_{i}")
        dfs.append(load_large_csv(os.path.join(data_dir, f'df_{i}.csv')))
        
        print(f"\nЗагрузка файла target_{i}")
        targets.append(load_large_csv(os.path.join(data_dir, f'target_{i}.csv')))

    combined_df = pd.concat(dfs, ignore_index=True)
    combined_target = pd.concat(targets, ignore_index=True)
    
    print("\nЗагрузка валидационных файлов")
    df_val = load_large_csv(os.path.join(data_dir, 'df_val.csv'))
    target_val = load_large_csv(os.path.join(data_dir, 'target_val.csv'))
    
    return combined_df, combined_target, df_val, target_val

In [80]:
data_dir = '/home/c5/Рабочий стол/Модуль 4/BIG DATA'
combined_df, combined_target, df_val, target_val = load_and_process_files(data_dir)


Загрузка файла df_0
Обрабатываем чанк 1 из файла df_0.csv
Обрабатываем чанк 2 из файла df_0.csv
Обрабатываем чанк 3 из файла df_0.csv
Обрабатываем чанк 4 из файла df_0.csv
Обрабатываем чанк 5 из файла df_0.csv
Обрабатываем чанк 6 из файла df_0.csv
Обрабатываем чанк 7 из файла df_0.csv
Обрабатываем чанк 8 из файла df_0.csv
Обрабатываем чанк 9 из файла df_0.csv
Обрабатываем чанк 10 из файла df_0.csv
Обрабатываем чанк 11 из файла df_0.csv
Обрабатываем чанк 12 из файла df_0.csv
Обрабатываем чанк 13 из файла df_0.csv
Обрабатываем чанк 14 из файла df_0.csv
Обрабатываем чанк 15 из файла df_0.csv
Обрабатываем чанк 16 из файла df_0.csv
Обрабатываем чанк 17 из файла df_0.csv
Обрабатываем чанк 18 из файла df_0.csv
Обрабатываем чанк 19 из файла df_0.csv
Обрабатываем чанк 20 из файла df_0.csv
Обрабатываем чанк 21 из файла df_0.csv
Обрабатываем чанк 22 из файла df_0.csv
Обрабатываем чанк 23 из файла df_0.csv
Обрабатываем чанк 24 из файла df_0.csv
Обрабатываем чанк 25 из файла df_0.csv
Обрабатываем 

In [4]:
gc.collect()

0

In [5]:
combined_df.head()

NameError: name 'combined_df' is not defined

In [6]:
combined_target.describe()

NameError: name 'combined_target' is not defined

In [94]:
combined_df.columns

Index(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12',
       '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24',
       '25', '26', '27', '28', '29'],
      dtype='object')